In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import os
import matplotlib.pyplot as plt
import torch

In [2]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Autenticazione “di servizio” fornita da Colab
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_id = '1gtE_lNw4MYiarzIu4qVAo-uu6aSzxS1d'
os.makedirs('/content/ndpi_files', exist_ok=True)

# Elenca e scarica tutti i file nella cartella
file_list = drive.ListFile(
    {'q': f"'{folder_id}' in parents and trashed=false"}
).GetList()

# 1) Definisci gli array
B = [
    "M-1.ndpi",
    "M-104.ndpi","M-105.ndpi","M-108.ndpi",
    "M-112.ndpi","M-121.ndpi","M-24.ndpi","M-30.ndpi",
    "M-32.ndpi","M-36.ndpi"
]
E = [
    "M-10.ndpi",
    "M-100.ndpi","M-102.ndpi","M-103.ndpi",
    "M-107.ndpi","M-109.ndpi","M-11.ndpi","M-110.ndpi",
    "M-111.ndpi","M-113.ndpi"
]
S = [
    "M-101.ndpi",
    "M-114.ndpi","M-65.ndpi","M-86.ndpi","M-87.ndpi"
]

base_dir = '/content/ndpi_files'
for sub in ('B','E','S'):
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

for f in file_list:
    title = f['title']
    if title in B:
        dest = os.path.join(base_dir, 'B', title)
    elif title in E:
        dest = os.path.join(base_dir, 'E', title)
    elif title in S:
        dest = os.path.join(base_dir, 'S', title)
    else:
        continue

    print(f"Scarico {title} in {os.path.dirname(dest)} …")
    f.GetContentFile(dest)

Scarico M-87.ndpi in /content/ndpi_files/S …
Scarico M-86.ndpi in /content/ndpi_files/S …
Scarico M-65.ndpi in /content/ndpi_files/S …
Scarico M-32.ndpi in /content/ndpi_files/B …
Scarico M-30.ndpi in /content/ndpi_files/B …
Scarico M-24.ndpi in /content/ndpi_files/B …
Scarico M-121.ndpi in /content/ndpi_files/B …
Scarico M-114.ndpi in /content/ndpi_files/S …


In [ ]:
!pip install openslide-python openslide-bin
import openslide

import glob

ndpi_path_b = glob.glob('/content/ndpi_files/B/*.ndpi')[0]
slide_b = openslide.OpenSlide(ndpi_path_b)
ndpi_path_e = glob.glob('/content/ndpi_files/E/*.ndpi')[0]
slide_e = openslide.OpenSlide(ndpi_path_e)
ndpi_path_s = glob.glob('/content/ndpi_files/S/*.ndpi')[0]
slide_s = openslide.OpenSlide(ndpi_path_s)

print("Dimensione B livello 0:", slide_b.level_dimensions[0])
print("Dimensione E livello 0:", slide_e.level_dimensions[0])
print("Dimensione S livello 0:", slide_s.level_dimensions[0])



In [ ]:
# Crea una miniatura con lato massimo 1024px
thumb = slide_b.get_thumbnail((1024, 1024))
plt.figure(figsize=(6,6))
plt.imshow(thumb)
plt.axis('off')
plt.title('Thumbnail livello 0 → ridotta a 1024×1024')
plt.show()


In [ ]:
!git clone https://github.com/LucaIanniello/MLIAProject.git

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

!conda env create -f MLIAProject/CLAM/env.yml
!conda install -n clam_latest -c conda-forge openslide
!conda install -n clam_latest -c conda-forge openslide-tools libopenslide0 libtiff-dev libjpeg-dev
!conda run -n clam_latest python --version
!conda run -n clam_latest pip list

!apt-get update
!apt-get install -y openslide-tools libopenslide0 libtiff-dev libjpeg-dev

In [ ]:

!conda --version
!which conda
!conda config --show-sources
!conda env update -n base -f MLIAProject/CLAM/env.yml


In [ ]:
!cp -r MLIAProject/presets .

In [ ]:
!MPLBACKEND=Agg conda run -n clam_latest python MLIAProject/CLAM/create_patches_fp.py \
    --source ndpi_files/B/ \
    --save_dir results/RESULTS_DIRECTORY_12_B \
    --patch_size 256 \
    --seg \
    --patch \
    --stitch \
    --preset process_list_edited_th12.csv



In [ ]:
!MPLBACKEND=Agg conda run -n clam_latest python MLIAProject/CLAM/create_patches_fp.py \
    --source ndpi_files/E/ \
    --save_dir results/RESULTS_DIRECTORY_12_E \
    --patch_size 256 \
    --seg \
    --patch \
    --stitch \
    --preset process_list_edited_th12.csv



In [ ]:
!MPLBACKEND=Agg conda run -n clam_latest python MLIAProject/CLAM/create_patches_fp.py \
    --source ndpi_files/S/ \
    --save_dir results/RESULTS_DIRECTORY_12_S \
    --patch_size 256 \
    --seg \
    --patch \
    --stitch \
    --preset process_list_edited_th12.csv



In [ ]:
!MPLBACKEND=Agg conda run -n clam_latest python -u MLIAProject/CLAM/extract_features_fp.py \
    --data_h5_dir "/content/results/RESULTS_DIRECTORY_12_B/" \
    --data_slide_dir "/content/ndpi_files/B/" \
    --csv_path "/content/results/RESULTS_DIRECTORY_12_B/process_list_autogen.csv" \
    --feat_dir results_features/FEATURES_DIRECTORY_12_B \
    --batch_size 128 \
    --slide_ext .ndpi

In [ ]:
!MPLBACKEND=Agg conda run -n clam_latest python -u MLIAProject/CLAM/extract_features_fp.py \
    --data_h5_dir "/content/results/RESULTS_DIRECTORY_12_E/" \
    --data_slide_dir "/content/ndpi_files/E/" \
    --csv_path "/content/results/RESULTS_DIRECTORY_12_E/process_list_autogen.csv" \
    --feat_dir results_features/FEATURES_DIRECTORY_12_E \
    --batch_size 128 \
    --slide_ext .ndpi

In [ ]:
!MPLBACKEND=Agg conda run -n clam_latest python -u MLIAProject/CLAM/extract_features_fp.py \
    --data_h5_dir "/content/results/RESULTS_DIRECTORY_12_S/" \
    --data_slide_dir "/content/ndpi_files/S/" \
    --csv_path "/content/results/RESULTS_DIRECTORY_12_S/process_list_autogen.csv" \
    --feat_dir results_features/FEATURES_DIRECTORY_12_S \
    --batch_size 128 \
    --slide_ext .ndpi

In [ ]:
import h5py

data = torch.load('results_features/FEATURES_DIRECTORY_12_B/pt_files/M-1.pt')
# Sostituisci 'nome_file.h5' con il percorso del tuo file .h5
with h5py.File('/content/results/RESULTS_DIRECTORY_12_B/patches/M-1.h5', 'r') as f:
    n_patches = f['coords'].shape[0]  # oppure f['coords'].shape[0]
    print(f"Numero di patch: {n_patches}")

print(type(data))
print(len(data))  # se è una lista
print(data.shape) # se è un tensore
print(data[:5])  # se è una lista o un tensore


In [ ]:
import h5py

data = torch.load('results_features/FEATURES_DIRECTORY_12_E/pt_files/M-10.pt')
# Sostituisci 'nome_file.h5' con il percorso del tuo file .h5
with h5py.File('/content/results/RESULTS_DIRECTORY_12_E/patches/M-10.h5', 'r') as f:
    n_patches = f['coords'].shape[0]  # oppure f['coords'].shape[0]
    print(f"Numero di patch: {n_patches}")


print(type(data))
print(len(data))  # se è una lista
print(data.shape) # se è un tensore
print(data[:5])  # se è una lista o un tensore

In [ ]:
import h5py

data = torch.load('results_features/FEATURES_DIRECTORY_12_S/pt_files/M-101.pt')
# Sostituisci 'nome_file.h5' con il percorso del tuo file .h5
with h5py.File('/content/results/RESULTS_DIRECTORY_12_S/patches/M-101.h5', 'r') as f:
    n_patches = f['coords'].shape[0]  # oppure f['coords'].shape[0]
    print(f"Numero di patch: {n_patches}")

print(type(data))
print(len(data))  # se è una lista
print(data.shape) # se è un tensore
print(data[:5])  # se è una lista o un tensore